In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [ ]:
df = pd.read_csv('s12old.csv')
df.head()

In [ ]:
df.tail()

In [ ]:


df = df.drop('Unnamed: 0', axis=1)


In [ ]:
df

In [ ]:


df = df.iloc[1001:]


In [ ]:
df

In [ ]:


df = df.iloc[0:].reset_index(drop=True)


In [ ]:
df

In [ ]:
import pandas as pd
import plotly.graph_objs as go

nn = df[20020:21020]

fig = go.Figure()
fig.add_trace(go.Scatter(x=nn['Frequency'], y=nn['S12'], mode='lines', name='S12'))



fig.update_layout(title='S12 vs. Frequency', xaxis_title='Frequency', yaxis_title='S12')

fig.show()

In [ ]:
import pandas as pd
import numpy as np

data = df

cf = []
g = []
h = []
j = []

for i in range(0, data.shape[0], 1001): 
    plot_data = data.iloc[i:i+1001]
    frequencies = plot_data['Frequency']
    s12_values = plot_data['S12']
    a = plot_data['ld1']
    b = plot_data['ld4']
    c = plot_data['dx']

    max_index = np.argmin(s12_values)

    ld1= np.median(a)
    ld4= np.median(b)
    dx= np.median(c)

    center_frequency = frequencies.iloc[max_index]

    cf.append(center_frequency)
    g.append(ld1)
    h.append(ld4)
    j.append(dx)

combined_list = [cf, g, h, j]
combined_df = pd.DataFrame(combined_list).T
combined_df.columns = ['Center Frequency', 'g', 'h', 'j']


combined_df


In [ ]:

row_with_least_cf = combined_df.iloc[combined_df['Center Frequency'].idxmin()]

print(row_with_least_cf)


In [ ]:

combined_df = combined_df.drop(235)


combined_df = combined_df.drop(186)



In [ ]:
combined_df.iloc[233]

In [ ]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X_train, X_test, y_train, y_test = train_test_split(combined_df[['g', 'h', 'j']], combined_df['Center Frequency'], test_size=0.2)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

print("R2:", r2)
print("MAE:", mae)
print("MSE:", mse)


In [ ]:


from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor

regressors = [
    ("Random Forest", RandomForestRegressor()),
    ("Support Vector Regressor", SVR()),
    ("K-Neighbors Regressor", KNeighborsRegressor()),
    ("Decision Tree Regressor", DecisionTreeRegressor()),
    ("Gradient Boosting Regressor", GradientBoostingRegressor()),
    ("AdaBoost Regressor", AdaBoostRegressor()),
    ("Bagging Regressor", BaggingRegressor()),
    ("Voting Regressor", VotingRegressor(estimators=[("lr", model), ("rf", RandomForestRegressor())])),
    ("XGBoost Regressor", XGBRegressor()),
]

for name, regressor in regressors:
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Regressor: {name}")
    print(f"R2: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print()


In [ ]:

fig = go.Figure()

fig.add_trace(go.Scatter(x=combined_df.index, y=combined_df['Center Frequency'], mode='lines', name='Center Frequency'))

fig.add_trace(go.Scatter(x=combined_df.index, y=XGBRegressor().fit(X_train, y_train).predict(combined_df[['g', 'h', 'j']]), mode='lines', name='XGBoost Prediction'))

fig.update_layout(title='Center Frequency and XGBoost Prediction', xaxis_title='Center Frequency', yaxis_title='Value')

fig.show()

In [ ]:


from sklearn.model_selection import GridSearchCV
param_grids = {
    "Random Forest": {
        "n_estimators": [100, 200, 300],
        "max_depth": [None, 5, 10],
    },
    "Support Vector Regressor": {
        "kernel": ["linear", "rbf"],
        "C": [1, 10, 100],
    },
    "K-Neighbors Regressor": {
        "n_neighbors": [3, 5, 7],
        "weights": ["uniform", "distance"],
    },
    "Decision Tree Regressor": {
        "criterion": ["squared_error"],
        "max_depth": [None, 5, 10],
    },
    "Gradient Boosting Regressor": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.1, 0.2, 0.3],
    },
    "AdaBoost Regressor": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.1, 0.2, 0.3],
    },
    "Bagging Regressor": {
        "n_estimators": [100, 200, 300],
    },
    "Voting Regressor": {
        "estimators": [
            ("lr", model),
            ("rf", RandomForestRegressor()),
        ],
        "weights": [0.5, 0.5],
    },
    "XGBoost Regressor": {
        "n_estimators": [100, 200, 300],
        "learning_rate": [0.1, 0.2, 0.3],
    },
}

for name, regressor in regressors:
    grid_search = GridSearchCV(regressor, param_grids[name], cv=5)
    grid_search.fit(X_train, y_train)

    best_estimator = grid_search.best_estimator_

    y_pred = best_estimator.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Regressor: {name}")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"R2: {r2}")
    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print()


In [ ]:

import shap

model = XGBRegressor()
model.fit(X_train, y_train)

explainer = shap.Explainer(model)

shap_values = explainer(X_test)

feature_importances = shap_values.values.mean(axis=0)

for i, feature in enumerate(combined_df[['g', 'h', 'j']].columns):
    print(f"{feature}: {feature_importances[i]}")


In [ ]:

import matplotlib.pyplot as plt

cf = combined_df['Center Frequency'].to_numpy()

plt.figure(figsize=(10, 6))
plt.scatter(range(len(cf)), cf, color='blue', label='Center Frequency')

plt.xlabel('Index', fontsize=14)
plt.ylabel('Center Frequency (GHz)', fontsize=14)
plt.title('Scatter Plot of Center Frequencies', fontsize=16)

plt.show()


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from scipy.optimize import minimize
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X = combined_df[['g', 'h', 'j']]
y = combined_df['Center Frequency']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = XGBRegressor()
rf_model.fit(X_train, y_train)

train_score = rf_model.score(X_train, y_train)
test_score = rf_model.score(X_test, y_test)
print(f"Train R^2: {train_score:.4f}")
print(f"Test R^2: {test_score:.4f}")

def surrogate_objective(dimensions):
    input_df = pd.DataFrame([dimensions], columns=['g', 'h', 'j'])
    predicted_frequency = rf_model.predict(input_df)[0]
    target_frequency = 6.2e9 
    return abs(predicted_frequency - target_frequency)

initial_guess = X.mean().values

result = minimize(surrogate_objective, initial_guess, method='Nelder-Mead')

optimal_dimensions = result.x
optimal_input = pd.DataFrame([optimal_dimensions], columns=['g', 'h', 'j'])
predicted_optimal_frequency = rf_model.predict(optimal_input)[0]

print("\nOptimization Results:")
print(f"Optimal dimensions (g, h, j): {optimal_dimensions}")
print(f"Predicted optimal frequency: {predicted_optimal_frequency:.2e} Hz")
print(f"Optimization success: {result.success}")
print(f"Number of iterations: {result.nit}")

feature_importance = rf_model.feature_importances_
feature_names = ['g', 'h', 'j']

plt.figure(figsize=(10, 6))
plt.bar(feature_names, feature_importance)
plt.title('Feature Importances')
plt.xlabel('Features')
plt.ylabel('Importance')
plt.show()

In [ ]:

import matplotlib.pyplot as plt
import numpy as np

from sklearn.inspection import permutation_importance

model = XGBRegressor()
model.fit(X_train, y_train)

permutation_importances = permutation_importance(model, X_test, y_test)

sorted_indices = np.argsort(-permutation_importances['importances_mean'])

for i in sorted_indices:
    feature_name = X_test.columns[i]
    importance = permutation_importances['importances_mean'][i] 
    print(f"{feature_name}: {importance:.2f}")

plt.figure()
plt.bar(X_test.columns[sorted_indices], permutation_importances['importances_mean'][sorted_indices])
plt.xlabel("Feature")
plt.ylabel("Importance")
plt.title("Permutation Importance")
plt.show()